### BERT

BERT, acrónimo de Bidirectional Encoder Representations from Transformers, es un modelo de aprendizaje profundo para el procesamiento del lenguaje natural (NLP) desarrollado por Google en 2018. BERT se basa en la arquitectura de transformadores y ha revolucionado el campo del NLP al proporcionar un enfoque bidireccional en el preentrenamiento de modelos de lenguaje. La principal ventaja de BERT radica en su capacidad para comprender el contexto completo de una palabra al considerar tanto el texto que la precede como el que la sigue, lo que contrasta con los modelos unidireccionales anteriores que solo podían tener en cuenta un contexto parcial.

Para entender BERT, es fundamental comprender la arquitectura de los transformers, en el artículo "Attention is All You Need" en 2017. Los transformadores son una arquitectura de red neuronal diseñada para manejar secuencias de datos, como texto natural, sin recurrir a estructuras recurrentes o convolucionales.

BERT utiliza solo la parte del codificador de la arquitectura de transformadores. A diferencia de los modelos de lenguaje tradicionales que leen el texto de izquierda a derecha o de derecha a izquierda, BERT emplea un enfoque bidireccional. Esto significa que en cada paso, BERT puede considerar tanto el contexto anterior como el posterior a la palabra en cuestión, lo que te permite capturar matices más complejos del lenguaje.

La implementación de BERT, como se observa en el código proporcionado, incluye varios componentes clave:

**Embedding**:

Transforma los IDs de tokens en vectores de embedding que contienen información sobre los tokens, sus posiciones y segmentos. Esto es esencial para que el modelo comprenda la estructura de las oraciones y las relaciones entre las palabras.

**Capas del codificador**:

Consisten en múltiples capas de atención y redes de alimentación directa, lo que permite al modelo capturar dependencias a largo plazo en la secuencia de entrada.

**Clasificador:**

Una red neuronal adicional que toma la salida del primer token ([CLS]) para tareas de clasificación, como determinar si una oración sigue lógicamente a otra.

**Decodificador:**

Compartido con la capa de embedding, este componente predice los tokens enmascarados durante el preentrenamiento, permitiendo al modelo aprender representaciones contextuales ricas.

**Preentrenamiento y Fine-Tuning:**

El entrenamiento de BERT se divide en dos fases:

Preentrenamiento:
- BERT se entrena en grandes corpus de texto sin etiquetas utilizando dos tareas: Modelado de Lenguaje Enmascarado (MLM) y Predicción de la Siguiente Oración (NSP). En MLM, ciertos tokens en la entrada se enmascaran y el modelo debe predecirlos, lo que obliga a BERT a comprender el contexto bidireccional. En NSP, el modelo aprende a predecir si dos oraciones en secuencia son coherentes.

Fine-Tuning:

- BERT se ajusta para tareas específicas de NLP (como clasificación de texto, etiquetado de entidades nombradas, y preguntas y respuestas) 
utilizando conjuntos de datos etiquetados más pequeños. Durante esta fase, todas las capas del modelo se entrenan conjuntamente en la tarea objetivo.


### Código de ejemplo 

El siguiente código implementa un modelo BERT (Bidirectional Encoder Representations from Transformers) para dos tareas de preentrenamiento clave: el Modelado de Lenguaje Enmascarado (MLM) y la Predicción de la Siguiente Oración (NSP). 

La clase `BERT` hereda de nn.Module y representa el modelo BERT, que se utiliza para tareas de procesamiento de lenguaje natural. BERT se basa en una arquitectura de transformers y es conocido por su capacidad para entender el contexto bidireccional en el texto.


**Embedding**:

- `self.embedding = Embedding()`: Esta instancia de la clase `Embedding` se encarga de convertir los IDs de los tokens en vectores de embedding que contienen información de posición y segmento.
Capas del Codificador:

- `self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])`: Una lista de capas del codificador (EncoderLayer). Cada una de estas capas incluye una atención multi-cabecera y una red neuronal de alimentación directa.

**Clasificador**:

- `self.fc = nn.Linear(d_model, d_model)`
- `self.activ1 = nn.Tanh()`
- `self.linear = nn.Linear(d_model, d_model)`
- `self.activ2 = gelu`
- `self.norm = nn.LayerNorm(d_model)`
- `self.classifier = nn.Linear(d_model, 2)`

Estas capas se utilizan para la clasificación final. `self.fc` y `self.linear` son capas lineales, `self.activ1` y `self.activ2` son funciones de activación, y `self.norm` es una capa de normalización.

**Decodificador**:

- `embed_weight = self.embedding.tok_embed.weight`
- `n_vocab, n_dim = embed_weight.size()`
- `self.decoder = nn.Linear(n_dim, n_vocab, bias=False)`
- `self.decoder.weight = embed_weight`
- `self.decoder_bias = nn.Parameter(torch.zeros(n_vocab))`

El decodificador comparte los pesos con la capa de embedding de tokens para predecir los tokens enmascarados. Esto ayuda a mantener la consistencia entre la entrada y la salida.

Para el método forward, se tiene lo siguiente: 

**Embeddings**:

- `output = self.embedding(input_ids, segment_ids)`: Convierte los `input_ids` y `segment_ids` en embeddings utilizando la capa de `Embedding`.

**Máscara de atención**:

- `enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids)`: Crea una máscara de atención para evitar que el modelo preste atención a los tokens de relleno (PAD).

**Capa del codificador**:

- `for layer in self.layers: output, enc_self_attn = layer(output, enc_self_attn_mask)`: Pasa las salidas a través de las capas del codificador, aplicando la atención y la red neuronal de alimentación directa en cada capa.

**Clasificación**:

- `h_pooled = self.activ1(self.fc(output[:, 0]))`: Utiliza la salida correspondiente al primer token ([CLS]) para la clasificación.
- `logits_clsf = self.classifier(h_pooled)`: Realiza la clasificación binaria.

**Predicción de tokens enmascarados**:

- `masked_pos = masked_pos[:, :, None].expand(-1, -1, output.size(-1))`: Expande las posiciones enmascaradas para que coincidan con la dimensión de la salida.
- `h_masked = torch.gather(output, 1, masked_pos)`: Recolecta las salidas en las posiciones enmascaradas.
- `h_masked = self.norm(self.activ2(self.linear(h_masked)))`: Aplica una capa lineal, una función de activación (GELU) y una capa de normalización.
- `logits_lm = self.decoder(h_masked) + self.decoder_bias`: Decodifica las salidas para predecir los tokens enmascarados.

**Salida**:

- `return logits_lm, logits_clsf`: Devuelve las predicciones de los tokens enmascarados (`logits_lm`) y las predicciones de clasificación de la oración (`logits_clsf`).


In [ ]:
import math
import re
from random import *
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Función para crear un batch de datos de entrada
def make_batch():
    batch = []
    positive = negative = 0
    while positive != batch_size / 2 or negative != batch_size / 2:
        # Selecciona índices aleatorios para las oraciones
        tokens_a_index, tokens_b_index = randrange(len(sentences)), randrange(len(sentences))
        tokens_a, tokens_b = token_list[tokens_a_index], token_list[tokens_b_index]
        input_ids = [word_dict['[CLS]']] + tokens_a + [word_dict['[SEP]']] + tokens_b + [word_dict['[SEP]']]
        segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)

        # Máscara LM
        n_pred = min(max_pred, max(1, int(round(len(input_ids) * 0.15)))) # 15 % de los tokens en una oración
        cand_maked_pos = [i for i, token in enumerate(input_ids)
                          if token != word_dict['[CLS]'] and token != word_dict['[SEP]']]
        shuffle(cand_maked_pos)
        masked_tokens, masked_pos = [], []
        for pos in cand_maked_pos[:n_pred]:
            masked_pos.append(pos)
            masked_tokens.append(input_ids[pos])
            if random() < 0.8:  # 80%
                input_ids[pos] = word_dict['[MASK]'] # hacer máscara
            elif random() < 0.5:  # 10%
                index = randint(0, vocab_size - 1) # índice aleatorio en el vocabulario
                input_ids[pos] = word_dict[number_dict[index]] # reemplazar

        # Relleno con ceros
        n_pad = maxlen - len(input_ids)
        input_ids.extend([0] * n_pad)
        segment_ids.extend([0] * n_pad)

        # Relleno con ceros para el resto de los tokens (100% - 15%)
        if max_pred > n_pred:
            n_pad = max_pred - n_pred
            masked_tokens.extend([0] * n_pad)
            masked_pos.extend([0] * n_pad)

        # Agregar ejemplos positivos y negativos al batch
        if tokens_a_index + 1 == tokens_b_index and positive < batch_size / 2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True]) # EsSiguiente
            positive += 1
        elif tokens_a_index + 1 != tokens_b_index and negative < batch_size / 2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False]) # NoEsSiguiente
            negative += 1
    return batch
# Procesamiento terminado

# Función para obtener la máscara de atención para los pads
def get_attn_pad_mask(seq_q, seq_k):
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) es el token PAD
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # batch_size x 1 x len_k(=len_q), uno es máscara
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # batch_size x len_q x len_k

# Función de activación gelu
def gelu(x):
    "Implementación de la función de activación gelu por Hugging Face"
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

# Clase de Embedding
class Embedding(nn.Module):
    def __init__(self):
        super(Embedding, self).__init__()
        self.tok_embed = nn.Embedding(vocab_size, d_model)  # embedding de tokens
        self.pos_embed = nn.Embedding(maxlen, d_model)  # embedding de posición
        self.seg_embed = nn.Embedding(n_segments, d_model)  # embedding de segmento (tipo de token)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x, seg):
        seq_len = x.size(1)
        pos = torch.arange(seq_len, dtype=torch.long)
        pos = pos.unsqueeze(0).expand_as(x)  # (seq_len,) -> (batch_size, seq_len)
        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
        return self.norm(embedding)

# Clase de Atención con Producto Escalar
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) # scores : [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        scores.masked_fill_(attn_mask, -1e9) # Llena elementos del tensor con valor donde la máscara es uno.
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V)
        return context, attn

# Clase de Atención Multi-Cabecera
class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_k * n_heads)
        self.W_K = nn.Linear(d_model, d_k * n_heads)
        self.W_V = nn.Linear(d_model, d_v * n_heads)
    def forward(self, Q, K, V, attn_mask):
        # q: [batch_size x len_q x d_model], k: [batch_size x len_k x d_model], v: [batch_size x len_k x d_model]
        residual, batch_size = Q, Q.size(0)
        # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # q_s: [batch_size x n_heads x len_q x d_k]
        k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # k_s: [batch_size x n_heads x len_k x d_k]
        v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # v_s: [batch_size x n_heads x len_k x d_v]

        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size x n_heads x len_q x len_k]

        # context: [batch_size x n_heads x len_q x d_v], attn: [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        context, attn = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads * d_v) # context: [batch_size x len_q x n_heads * d_v]
        output = nn.Linear(n_heads * d_v, d_model)(context)
        return nn.LayerNorm(d_model)(output + residual), attn # output: [batch_size x len_q x d_model]

# Clase de Red Neuronal de Alimentación Directa Posicional
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        # (batch_size, len_seq, d_model) -> (batch_size, len_seq, d_ff) -> (batch_size, len_seq, d_model)
        return self.fc2(gelu(self.fc1(x)))

# Clase de Capa de Codificador
class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, enc_inputs, enc_self_attn_mask):
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs a Q,K,V iguales
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size x len_q x d_model]
        return enc_outputs, attn

# Clase BERT (Bidirectional Encoder Representations from Transformers)
class BERT(nn.Module):
    def __init__(self):
        super(BERT, self).__init__()
        self.embedding = Embedding()
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])
        self.fc = nn.Linear(d_model, d_model)
        self.activ1 = nn.Tanh()
        self.linear = nn.Linear(d_model, d_model)
        self.activ2 = gelu
        self.norm = nn.LayerNorm(d_model)
        self.classifier = nn.Linear(d_model, 2)
        # El decodificador se comparte con la capa de embedding
        embed_weight = self.embedding.tok_embed.weight
        n_vocab, n_dim = embed_weight.size()
        self.decoder = nn.Linear(n_dim, n_vocab, bias=False)
        self.decoder.weight = embed_weight
        self.decoder_bias = nn.Parameter(torch.zeros(n_vocab))

    def forward(self, input_ids, segment_ids, masked_pos):
        output = self.embedding(input_ids, segment_ids)
        enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids)
        for layer in self.layers:
            output, enc_self_attn = layer(output, enc_self_attn_mask)
        # output : [batch_size, len, d_model], attn : [batch_size, n_heads, d_mode, d_model]
        # Se decide por el primer token (CLS)
        h_pooled = self.activ1(self.fc(output[:, 0])) # [batch_size, d_model]
        logits_clsf = self.classifier(h_pooled) # [batch_size, 2]

        masked_pos = masked_pos[:, :, None].expand(-1, -1, output.size(-1)) # [batch_size, max_pred, d_model]
        # Obtener la posición enmascarada de la salida final del transformer.
        h_masked = torch.gather(output, 1, masked_pos) # posición enmascarada [batch_size, max_pred, d_model]
        h_masked = self.norm(self.activ2(self.linear(h_masked)))
        logits_lm = self.decoder(h_masked) + self.decoder_bias # [batch_size, max_pred, n_vocab]

        return logits_lm, logits_clsf

if __name__ == '__main__':
    # Parámetros de BERT
    maxlen = 30 # longitud máxima
    batch_size = 6
    max_pred = 5  # máximo de tokens de predicción
    n_layers = 6 # número de codificadores en la capa del codificador
    n_heads = 12 # número de cabeceras en la atención multi-cabecera
    d_model = 768 # tamaño del embedding
    d_ff = 768 * 4  # 4*d_model, dimensión de FeedForward
    d_k = d_v = 64  # dimensión de K(=Q), V
    n_segments = 2

    text = (
        'Hello, how are you? I am Romeo.\n'
        'Hello, Romeo My name is Juliet. Nice to meet you.\n'
        'Nice meet you too. How are you today?\n'
        'Great. My baseball team won the competition.\n'
        'Oh Congratulations, Juliet\n'
        'Thanks you Romeo'
    )
    sentences = re.sub("[.,!?\\-]", '', text.lower()).split('\n')  # filtrar '.', ',', '?', '!'
    word_list = list(set(" ".join(sentences).split()))
    word_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}
    for i, w in enumerate(word_list):
        word_dict[w] = i + 4
    number_dict = {i: w for i, w in enumerate(word_dict)}
    vocab_size = len(word_dict)

    token_list = list()
    for sentence in sentences:
        arr = [word_dict[s] for s in sentence.split()]
        token_list.append(arr)

    model = BERT()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    batch = make_batch()
    input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

    for epoch in range(100):
        optimizer.zero_grad()
        logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
        loss_lm = criterion(logits_lm.transpose(1, 2), masked_tokens) # para LM enmascarado
        loss_lm = (loss_lm.float()).mean()
        loss_clsf = criterion(logits_clsf, isNext) # para clasificación de oraciones
        loss = loss_lm + loss_clsf
        if (epoch + 1) % 10 == 0:
            print('Epoca:', '%04d' % (epoch + 1), 'costo =', '{:.6f}'.format(loss))
        loss.backward()
        optimizer.step()

    # Predecir tokens enmascarados y esSiguiente
    input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(batch[0]))
    print(text)
    print([number_dict[w.item()] for w in input_ids[0] if number_dict[w.item()] != '[PAD]'])

    logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
    logits_lm = logits_lm.data.max(2)[1][0].data.numpy()
    print('lista de tokens enmascarados : ',[pos.item() for pos in masked_tokens[0] if pos.item() != 0])
    print('lista de tokens enmascarados predichos : ',[pos for pos in logits_lm if pos != 0])

    logits_clsf = logits_clsf.data.max(1)[1].data.numpy()[0]
    print('esNext : ', True if isNext else False)
    print('predecir esNext : ',True if logits_clsf else False)


El código crea un conjunto de datos donde se combinan pares de oraciones para formar secuencias de entrada que el modelo BERT utilizará para el entrenamiento. A continuación se muestra un ejemplo de una secuencia de tokens generada:

In [ ]:
#['[CLS]', 'hello', '[MASK]', 'are', 'you', 'i', 'am', 'hello', '[SEP]', 'thanks', 'you', 'romeo', '[SEP]']

Aquí:

- [CLS] es el token de clasificación utilizado al principio de cada secuencia.
- [SEP] es el token separador que se utiliza para dividir dos oraciones diferentes dentro de la misma secuencia.
- [MASK] es el token de máscara que el modelo debe predecir durante el entrenamiento.

La lista de tokens enmascarados y sus posiciones son las siguientes:

In [ ]:
#lista de tokens enmascarados : [21, 20]


Esto indica que se enmascararon dos posiciones en la secuencia original. En este caso, el token en la posición 21 fue enmascarado.

In [ ]:
#lista de tokens enmascarados predichos : [20, 20]

Esto significa que el modelo predijo incorrectamente el token en la posición enmascarada, repitiendo el mismo token dos veces en lugar de encontrar el token correcto.

La clasificación de si la segunda oración sigue lógicamente a la primera es indicada por los valores de `esNext` y `predecir esNext`:

In [ ]:
#esNext : False
#predecir esNext : True

Aquí, `esNext` : `False` significa que, en el conjunto de datos original, las dos oraciones no son secuenciales (no están una después de la otra en el texto original). Sin embargo, predecir `esNext : True` muestra que el modelo BERT predijo incorrectamente que las oraciones eran secuenciales.

#### Ejercicios: 

1 . Ajusta el preprocesamiento de datos para manejar un nuevo conjunto de datos y experimentar con diferentes técnicas de tokenización.

Tareas:
- Cambia el texto de entrada por un nuevo corpus de datos.
- Implementa una nueva técnica de tokenización (por ejemplo, tokenización basada en subpalabras usando Byte Pair Encoding).
- Ajusta el código para que utilice el nuevo tokenizador y diccionario de palabras.

2 . Experimentar con diferentes hiperparámetros para observar su impacto en el rendimiento del modelo.

Tareas:
- Cambia los parámetros del modelo como `n_layers`, `n_heads`, `d_model`, `d_ff`.
- Realiza entrenamientos con diferentes configuraciones y registra el costo del modelo durante el entrenamiento.
- Compara el rendimiento en términos de pérdida y precisión para diferentes configuraciones de parámetros.

3  Implementación de una nueva función de pérdida

Tareas:

- Implementa la pérdida de Kullback-Leibler Divergence en lugar de CrossEntropyLoss para la tarea de MLM.
- Ajusta el código de entrenamiento para utilizar la nueva función de pérdida.
- Compara el rendimiento del modelo utilizando la nueva función de pérdida.

4 . Realiza fine-tuning del modelo BERT en una tarea específica de NLP, como clasificación de texto o reconocimiento de entidades nombradas.

Tareas:
- Selecciona un conjunto de datos para una tarea específica (por ejemplo, IMDB dataset para clasificación de sentimientos).
- Ajusta el modelo BERT y el código de entrenamiento para realizar fine-tuning en la nueva tarea.
- Entrena el modelo y evalúa su rendimiento en la tarea seleccionada.

5 . Visualiza las matrices de atención para entender cómo el modelo BERT presta atención a diferentes partes de la secuencia de entrada.

Tareas:
- Modifica el código para almacenar las matrices de atención durante la inferencia.
- Utiliza una librería de visualización (por ejemplo, matplotlib o seaborn) para visualizar las matrices de atención.
- Analiza y comenta las visualizaciones para entender mejor el comportamiento del modelo.

6 .Implementa técnicas de regularización para prevenir el sobreajuste durante el entrenamiento.

Tareas:
- Añade Dropout a las capas del modelo BERT.
- Ajusta los hiperparámetros del Dropout y realiza entrenamientos con diferentes tasas de Dropout.
- Evalúa el impacto de la regularización en el rendimiento del modelo.

7 . Utiliza un modelo preentrenado BERT de Hugging Face y realizar fine-tuning en una nueva tarea.

Tareas:

- Carga un modelo preentrenado BERT utilizando la librería transformers de Hugging Face.
- Realiza fine-tuning del modelo en una tarea específica (por ejemplo, MRPC o CoLA).
- Evalúa y compara el rendimiento del modelo preentrenado con el modelo entrenado desde cero.

In [ ]:
# Tus respuestas

### Variantes de BERT

Desde su introducción en 2018, BERT (Bidirectional Encoder Representations from Transformers) ha revolucionado el campo del procesamiento de lenguaje natural (NLP). Su capacidad para capturar contextos bidireccionales ha permitido mejoras significativas en diversas tareas de NLP. A raíz del éxito de BERT, se han desarrollado múltiples variantes que optimizan diferentes aspectos del modelo original, como la eficiencia, la capacidad de generalización y la adecuación a contextos específicos. Veamos algunas de las variantes más destacadas de BERT, incluyendo RoBERTa, DistilBERT, ALBERT, TinyBERT y otras adaptaciones especializadas.

**RoBERTa (Robustly Optimized BERT Pretraining Approach)**

[RoBERTa](https://arxiv.org/abs/1907.11692), introducido por Facebook AI, es una mejora sobre el modelo BERT original. Este modelo se basa en la misma arquitectura de transformadores, pero optimiza el preentrenamiento eliminando la tarea de predicción de la siguiente oración (Next Sentence Prediction, NSP). En su lugar, RoBERTa se entrena con secuencias más largas y en volúmenes de datos significativamente mayores.

Características principales de RoBERTa:

- Eliminación de NSP: La eliminación de la tarea NSP permitió a RoBERTa enfocarse exclusivamente en el modelado de lenguaje enmascarado (MLM), lo que mejoró la eficiencia y el rendimiento.
- Aumento de datos de entrenamiento: RoBERTa se entrenó en un corpus de datos diez veces mayor que el utilizado por BERT, incluyendo datos adicionales de libros, Wikipedia y otros recursos.
- Tamaño de secuencia: Utiliza secuencias de texto más largas (hasta 512 tokens) durante el entrenamiento, lo que mejora la capacidad del modelo para capturar dependencias a largo plazo.
- Ajustes de hiperparámetros: Se realizaron ajustes finos en los hiperparámetros de entrenamiento, como el tamaño del batch y la tasa de aprendizaje, para mejorar la eficacia del modelo.

RoBERTa ha demostrado un rendimiento superior en varias tareas de NLP, superando consistentemente a BERT en benchmarks estándar como GLUE, RACE y SQuAD.

**DistilBERT**

Desarrollado por Hugging Face, [DistilBERT](https://arxiv.org/abs/1910.01108) es una versión más pequeña y eficiente de BERT. Utiliza técnicas de destilación de conocimientos para reducir el tamaño del modelo en aproximadamente un 40%, manteniendo el 97% del rendimiento de BERT en diversas tareas de NLP.

Características principales de DistilBERT:

- Destilación de conocimientos: El proceso de destilación implica entrenar un modelo más pequeño (el estudiante) para imitar el comportamiento de un modelo más grande (el maestro), en este caso, BERT.
- Reducción de parámetros: DistilBERT tiene menos capas (6 en lugar de 12), lo que reduce significativamente el número de parámetros y la complejidad computacional.
- Velocidad y eficiencia: Al ser más compacto, DistilBERT es más rápido y menos costoso en términos de recursos computacionales, lo que lo hace ideal para aplicaciones con limitaciones de hardware.

DistilBERT es especialmente útil para aplicaciones móviles y de tiempo real, donde la latencia y el consumo de recursos son críticos.

**ALBERT (A Lite BERT)**

[ALBERT](https://www.arxiv.org/abs/1909.11942), desarrollado por Google Research, es una versión ligera de BERT que introduce varias técnicas innovadoras para reducir el tamaño del modelo sin sacrificar el rendimiento.

Características principales de ALBERT:

- Factorización de embeddings: ALBERT reduce el tamaño de los embeddings de palabras al factorizar la matriz de embeddings en dos matrices más pequeñas. Esto reduce el número de parámetros y mejora la eficiencia.
- Parámetros compartidos: Utiliza parámetros compartidos entre las capas del codificador, lo que reduce aún más el número de parámetros sin afectar la capacidad de representación del modelo.
- Mejora en el preentrenamiento: ALBERT introduce una nueva tarea de predicción de la ordenación de oraciones (Sentence Order Prediction, SOP) en lugar de NSP, lo que mejora la capacidad del modelo para comprender la coherencia del texto.

ALBERT ha demostrado un rendimiento comparable al de BERT en benchmarks estándar con un costo computacional significativamente menor.

**TinyBERT**

[TinyBERT](https://arxiv.org/abs/1909.10351) es otra variante optimizada para la eficiencia, desarrollada mediante técnicas de destilación de conocimientos. Este modelo es aún más pequeño que DistilBERT y está diseñado para ser altamente eficiente sin comprometer demasiado el rendimiento.

Características principales de TinyBERT:

- Destilación de dos etapas: TinyBERT utiliza un proceso de destilación en dos etapas que incluye la destilación general en un corpus grande y la destilación específica para una tarea particular.
- Reducción de tamaño: TinyBERT es extremadamente compacto, con menos parámetros y una estructura más simple, lo que lo hace ideal para dispositivos con recursos limitados.
- Rendimiento competitivo: A pesar de su pequeño tamaño, TinyBERT mantiene un rendimiento competitivo en varias tareas de NLP.

TinyBERT es adecuado para aplicaciones donde la eficiencia y la velocidad son cruciales, como en dispositivos móviles y aplicaciones de tiempo real.










### Ejercicios

1 . Implementa una variante de BERT, como DistilBERT o ALBERT, y comparar su rendimiento con el modelo original.

Tareas:
- Implementa DistilBERT o ALBERT basándote en la arquitectura del modelo BERT original.
- Ajusta el código de entrenamiento para entrenar la nueva variante del modelo.
- Compara el rendimiento y la eficiencia de la variante con el modelo BERT original en términos de precisión, tamaño del modelo y tiempo de entrenamiento.

2 . Evalua el rendimiento del modelo entrenado en un conjunto de datos completamente nuevo y no visto durante el entrenamiento.

Tareas:
- Prepara un nuevo conjunto de datos para la evaluación.
- Ajusta el código de evaluación para utilizar el nuevo conjunto de datos.
- Evalúa el rendimiento del modelo y analiza las predicciones.

3 . Optimiza el código de entrenamiento para mejorar la eficiencia computacional y reducir el tiempo de entrenamiento.

Tareas:
- Identifica posibles cuellos de botella en el código de entrenamiento.
- Implementa optimizaciones como el uso de mixed precision training o gradient accumulation.
- Mide el tiempo de entrenamiento antes y después de las optimizaciones y compara los resultados.

In [ ]:
## Tus respuestas

### Otras adaptaciones y variantes de BERT

Además de las variantes mencionadas, existen otras adaptaciones de BERT que están diseñadas para contextos específicos y tareas especializadas:

**BERTweet**:

[BERTweet](https://aclanthology.org/2020.emnlp-demos.2.pdf) es una adaptación de BERT entrenada específicamente en datos de Twitter. Este modelo está optimizado para comprender y generar texto en el estilo y contexto particular de las redes sociales.

**BioBERT**:

[BioBERT](https://arxiv.org/abs/1901.08746) está entrenado en grandes corpus de texto biomédico y está diseñado para tareas en el dominio de la biomedicina y la biología, como la extracción de información biomédica y el reconocimiento de entidades nombradas en texto científico.

**SciBERT**:

[SciBERT](https://arxiv.org/abs/1903.10676) es una variante de BERT entrenada en artículos científicos de varias disciplinas. Está optimizado para tareas de NLP en el contexto de la literatura científica, como la extracción de términos técnicos y la clasificación de texto académico.

**Multilingual BERT (mBERT)**:

[mBERT](https://aclanthology.org/P19-1493/) es una versión de BERT entrenada en múltiples idiomas. Este modelo es capaz de manejar tareas de NLP en varios idiomas sin necesidad de entrenamientos adicionales específicos para cada lengua.

**SpanBERT**:

[SpanBERT](https://arxiv.org/abs/1907.10529) es una mejora sobre BERT que se enfoca en el modelado de spans (fragmentos de texto) en lugar de palabras individuales. Este modelo está diseñado para mejorar el rendimiento en tareas que involucran la predicción de relaciones entre spans de texto, como la extracción de relaciones y la respuesta a preguntas.


### Ejercicios

1 .Entrena BERTweet para una tarea específica de análisis de sentimientos en datos de Twitter.

Tareas:

- Descarga un conjunto de datos de tweets etiquetados con sentimientos (por ejemplo, Sentiment140).
- Carga el modelo preentrenado BERTweet utilizando la librería transformers de Hugging Face.
- Ajusta el código de entrenamiento para realizar fine-tuning en el conjunto de datos de análisis de sentimientos.
- Evalúa el rendimiento del modelo en un conjunto de datos de prueba y analiza las predicciones.

2 . Utiliza BioBERT para la tarea de reconocimiento de entidades nombradas (NER) en el dominio biomédico.

Tareas:
- Descarga un conjunto de datos etiquetados con entidades biomédicas (por ejemplo, BC5CDR).
- Carga el modelo preentrenado BioBERT.
- Ajusta el código de entrenamiento para realizar fine-tuning en la tarea de NER biomédico.
- Evalúa el rendimiento del modelo y analiza la precisión de las entidades reconocidas.

3 . Utiliza SciBERT para la tarea de clasificación de artículos científicos en diferentes categorías.

Tareas:
- Descarga un conjunto de datos de artículos científicos etiquetados por categorías (por ejemplo, arXiv dataset).
- Carga el modelo preentrenado SciBERT.
- Ajusta el código de entrenamiento para realizar fine-tuning en la tarea de clasificación de artículos científicos.
- Evalúa el rendimiento del modelo y analiza las predicciones de categorías.

4 .Utiliza mBERT para la tarea de reconocimiento de entidades nombradas en múltiples idiomas.

Tareas:
- Descarga conjuntos de datos de NER en varios idiomas (por ejemplo, CoNLL-2002 y CoNLL-2003).
- Carga el modelo preentrenado mBERT.
- Ajusta el código de entrenamiento para realizar fine-tuning en las tareas de NER en diferentes idiomas.
- Evalúa el rendimiento del modelo en cada idioma y compara los resultados.

5 .Utiliza SpanBERT para la tarea de extracción de relaciones entre entidades en textos.

Tareas:

- Descarga un conjunto de datos de extracción de relaciones (por ejemplo, TACRED).
- Carga el modelo preentrenado SpanBERT.
- Ajusta el código de entrenamiento para realizar fine-tuning en la tarea de extracción de relaciones.
- Evalúa el rendimiento del modelo y analiza la precisión de las relaciones extraídas.

6 .Compara el rendimiento de diferentes variantes de BERT en una tarea común.

Tareas:
- Selecciona una tarea común de NLP (por ejemplo, clasificación de texto).
- Carga los modelos preentrenados BERT, BERTweet, BioBERT, SciBERT, mBERT y SpanBERT.
- Realiza fine-tuning de cada modelo en el mismo conjunto de datos.
- Evalúa y compara el rendimiento de los modelos en términos de precisión, recall y F1-score.

7 . Visualizar las matrices de atención de diferentes variantes de BERT para entender cómo cada modelo presta atención a diferentes partes del texto.

Tareas:
- Selecciona un texto de entrada y aplica diferentes variantes de BERT (BERT, BERTweet, BioBERT, SciBERT, mBERT, SpanBERT).
- Modifica el código para almacenar las matrices de atención de cada modelo durante la inferencia.
- Utiliza una librería de visualización (por ejemplo, matplotlib o seaborn) para visualizar las matrices de atención.
- Analiza y comenta las visualizaciones para entender mejor las diferencias en el comportamiento de los modelos.

8 . Adapta una variante de BERT a un nuevo dominio específico utilizando un pequeño conjunto de datos etiquetados.

Tareas:
- Selecciona un dominio específico (por ejemplo, legal, financiero).
- Recopila un pequeño conjunto de datos etiquetados en el dominio seleccionado.
- Carga una variante de BERT adecuada (por ejemplo, SciBERT para textos científicos).
- Realiza fine-tuning del modelo en el nuevo conjunto de datos.
- Evalúa el rendimiento del modelo en el nuevo dominio y analiza las predicciones.

In [ ]:
## Sugerencia

#pip install transformers torch datasets
from datasets import load_dataset

# Cargar el conjunto de datos Sentiment140
dataset = load_dataset('sentiment140')

from transformers import AutoTokenizer

# Cargar el tokenizador BERTweet
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")

def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

# Aplicar el tokenizador a los datos de entrenamiento y prueba
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Remover columnas no utilizadas y establecer el formato de los tensores
tokenized_datasets = tokenized_datasets.remove_columns(['text'])
tokenized_datasets.set_format('torch')

from torch.utils.data import DataLoader

batch_size = 16

train_dataloader = DataLoader(tokenized_datasets['train'], batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(tokenized_datasets['test'], batch_size=batch_size)

from transformers import AutoModelForSequenceClassification

# Cargar el modelo BERTweet
model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=2)

from transformers import AdamW
from tqdm.auto import tqdm

# Configurar el optimizador
optimizer = AdamW(model.parameters(), lr=5e-5)

# Mover el modelo a la GPU si está disponible
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# Función de entrenamiento
def train():
    model.train()
    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['sentiment'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Función de evaluación
def evaluate():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['sentiment'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            _, predicted = torch.max(outputs.logits, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    print(f'Exactitud: {accuracy * 100:.2f}%')

# Entrenamiento del modelo
epochs = 3
for epoch in range(epochs):
    print(f'Epoca {epoch + 1}/{epochs}')
    train()
    evaluate()

evaluate()


In [ ]:
## Tus respuestas